In [1]:
import numpy as np
import h5py
from scipy.io import loadmat

In [2]:
def Patch(data,height_index,width_index,PATCH_SIZE):   # PATCH_SIZE为一个patch（边长-1）的一半    data维度(H,W,C)
    height_slice = slice(height_index-PATCH_SIZE, height_index+PATCH_SIZE)
    width_slice = slice(width_index-PATCH_SIZE, width_index+PATCH_SIZE)
    # 由height_index和width_index定位patch中心像素
    patch = data[height_slice, width_slice,:]
    patch = patch.reshape(-1,patch.shape[0]*patch.shape[1]*patch.shape[2])
    # print(patch.shape) 
    return patch

In [3]:
img = h5py.File('../../dataset/4_Chikusei/Chikusei_MATLAB/HyperspecVNIR_Chikusei_20140729.mat')['chikusei']
gt = loadmat('../../dataset/4_Chikusei/Chikusei_MATLAB/HyperspecVNIR_Chikusei_20140729_Ground_Truth.mat')['GT'][0][0][0]
img = np.array(img).transpose((2, 1, 0))
print(img.shape) 
'''
(2517, 2335, 128)
'''

print(gt.max())

'''19'''

(2517, 2335, 128)
19


'19'

In [4]:
'''
波段处理
'''
bands_select = [10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 36, 37, 38, 39, 40, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128]
for i in range(len(bands_select)):
    bands_select[i] = bands_select[i] - 1 
# print(bands_select)

'''
波段选择
'''
img = img[:, :, bands_select]
print(img.shape)
'''(2517, 2335, 100)'''
'''
图像归一化
'''
img = ( img * 1.0 - img.min() ) / ( img.max() - img.min() )
type(img)

(2517, 2335, 100)


numpy.ndarray

In [5]:
[m, n, b] = img.shape
label_num = gt.max()
print(label_num)
''' 19 除了0之外有19类'''
PATCH_SIZE = 14

19


In [6]:
# padding the hyperspectral images
img_temp = np.zeros((m + 2 * PATCH_SIZE, n + 2 * PATCH_SIZE, b), dtype=np.float32)
img_temp[PATCH_SIZE:(m + PATCH_SIZE), PATCH_SIZE:(n + PATCH_SIZE), :] = img[:, :, :]

for i in range(PATCH_SIZE):
    img_temp[i, :, :] = img_temp[2 * PATCH_SIZE - i, :, :]
    img_temp[m + PATCH_SIZE + i, :, :] = img_temp[m + PATCH_SIZE - i - 2, :, :]

for i in range(PATCH_SIZE):
    img_temp[:, i, :] = img_temp[:, 2 * PATCH_SIZE - i, :]
    img_temp[:, n + PATCH_SIZE + i, :] = img_temp[:, n + PATCH_SIZE  - i - 2, :]

img = img_temp
del img_temp

gt_temp = np.zeros((m + 2 * PATCH_SIZE, n + 2 * PATCH_SIZE), dtype=np.int8)
gt_temp[PATCH_SIZE:(m + PATCH_SIZE), PATCH_SIZE:(n + PATCH_SIZE)] = gt[:, :]
gt = gt_temp
del gt_temp

In [7]:
print(img.shape)
print(gt.shape)
'''
(2545, 2363, 100)
(2545, 2363)
'''

(2545, 2363, 100)
(2545, 2363)


'\n(2545, 2363, 100)\n(2545, 2363)\n'

In [8]:
[m, n, b] = img.shape

data = []
label = []
count = 0

In [9]:
for i in range(PATCH_SIZE, m - PATCH_SIZE):
    for j in range(PATCH_SIZE, n - PATCH_SIZE):
        if gt[i, j] == 0:
            continue
        else:
            count += 1
            temp_data = Patch(img, i, j, PATCH_SIZE)
            # temp_label = np.zeros((1, label_num), dtype=np.int8)  # temp_label为一行九列[0,1,2,....,7,8]表示类别
            temp_label = gt[i, j] - 1

            data.append(temp_data)  # 每一行表示一个patch
            label.append(temp_label)


print(count)  
'''
77592
'''

77592


'\n77592\n'

In [10]:
print(type(data), len(data))

<class 'list'> 77592


In [11]:
data = np.asarray(data)
print(data.shape) 
# (77592, 1, 78400)

data = np.squeeze(data)
print("squeeze : ", data.shape)
# squeeze :  (77592, 78400)

label = np.asarray(label)
print(label.shape)
# (77592,)

label = np.squeeze(label)
print("squeeze : ", label.shape)
# squeeze :  (77592,)

(77592, 1, 78400)
squeeze :  (77592, 78400)
(77592,)
squeeze :  (77592,)


In [12]:
print(type(label))
ch_dict = {}
ch_dict = ch_dict.fromkeys([i for i in range(label_num)], 0)
print(ch_dict)

for i in label:
    if i in ch_dict:
        ch_dict[i] += 1
print(ch_dict)
ch_label_sum = sum(ch_dict.values())
print(ch_label_sum)

<class 'numpy.ndarray'>
{0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0}
{0: 2845, 1: 2859, 2: 286, 3: 4852, 4: 4297, 5: 1108, 6: 20516, 7: 6515, 8: 13369, 9: 1268, 10: 5961, 11: 2193, 12: 1220, 13: 7664, 14: 431, 15: 222, 16: 1040, 17: 801, 18: 145}
77592


In [14]:
indices = np.arange(data.shape[0])  # list [0,.....,42775]
shuffled_indices = np.random.permutation(indices)
images = data[shuffled_indices]
labels = label[shuffled_indices]  # 打乱顺序

y = labels  # 布尔索引
# y 为一个list (42776,)  [0 1 6 ... 0 7 1]

n_classes = y.max() + 1  # y.max() = 8  n_classes代表类别数为9
t_labeled = []

In [15]:
# 改变类别数量---------------------------------------------------！！！！
for c in range(n_classes-1):  # 第19类样本数量小于200
    i = indices[y == c][:200]
    t_labeled += list(i)  # 列表中元素增加

In [16]:
# 将其划分分训练和检验两个数据集
t_images = images[t_labeled]
print('t_images', t_images.shape)
t_labels = labels[t_labeled]
print('t_labels', t_labels.shape)
# t_images (3600, 78400)
# t_labels (3600,)

t_images (3600, 78400)
t_labels (3600,)


In [17]:
f = h5py.File(r'../../h5dataset_ica_bandselect_200/CH-' + str(PATCH_SIZE * 2) + '-' + str(PATCH_SIZE * 2) + '-100.h5', 'w')  # 每类200个
f['data'] = t_images
f['label'] = t_labels
f.close()


f = h5py.File(r'../../h5dataset_ica_bandselect/CH-' + str(PATCH_SIZE * 2) + '-' + str(PATCH_SIZE * 2) + '-100.h5', 'w')  # 每类200个
f['data'] = images
f['label'] = labels
f.close()

print("success")

success


In [18]:
#HDF5的读取
f = h5py.File('../../h5dataset_ica_bandselect/CH-28-28-100.h5','r')   #打开h5文件  # 可以查看所有的主键  
# print(type(f))

print([key for key in f.keys()])

# print('first, we get values of x:', f['data'][:])
# print('then, we get values of y:', f['label'][:])
print(f['data'][:].shape)
print(f['label'][:].shape)

['data', 'label']


MemoryError: Unable to allocate 22.7 GiB for an array with shape (77592, 78400) and data type float32

In [19]:
#HDF5的读取
f = h5py.File('../../h5dataset_ica_bandselect_200/CH-28-28-100.h5','r')   #打开h5文件  # 可以查看所有的主键  
# print(type(f))

print([key for key in f.keys()])

# print('first, we get values of x:', f['data'][:])
# print('then, we get values of y:', f['label'][:])
print(f['data'][:].shape)
print(f['label'][:].shape)

['data', 'label']
(3600, 78400)
(3600,)


In [20]:
#HDF5的读取
f = h5py.File('../../h5dataset/CH-28-28-100.h5','r')   #打开h5文件  # 可以查看所有的主键  
# print(type(f))

print([key for key in f.keys()])

# print('first, we get values of x:', f['data'][:])
# print('then, we get values of y:', f['label'][:])
print(f['data'][:].shape)
print(f['label'][:].shape)

['data', 'label']
(3600, 78400)
(3600,)
